# 라이브러리 import

In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import json
import time
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotVisibleException
import pandas as pd 

# 리뷰 페이지 접속

리뷰페이지 접속 방법
- 자신이 찾을 지점의 고유번호를 입력하기
- 고유번호 찾는 방법: 
    1. 네이버맵 사이트에서 [검색할 지점] 검색
    2. 바 형태로 뜬 지점 정보에 마우스 우클릭
    3. 프레임소스 보기 클릭
    4. 소스보기 화면 url에서 [지점 번호] 찾기
    ex. view-source:https://pcmap.place.naver.com/restaurant/[지점번호]/home?from=map&fromPanelNum=1&ts=1633655558076
    
- 리뷰페이지 url 예시 : https://pcmap.place.naver.com/restaurant/+[지점번호]+/home?from=map&amp;fromPanelNum=2&amp

## 가게 영업정보 크롤링(민주)

## 방문자 리뷰 탭에서 크롤링

In [61]:
#홈에서의 더보기 경로는 홈페이지마다 위치가 달라지므로 f12 버튼 눌러서 파악해야 함
def scroll_click(review_driver, see_more_xpath):
    #홈에서 스크롤 다운
    review_driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    #홈에서 방문자 리뷰 더보기 클릭
    review_driver.find_element_by_xpath(see_more_xpath).click()
    #더보기 탭 끝까지 클릭하기 
    while True:
        try:
            review_driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[5]/div[4]/div[5]/div[2]/a').click()
        except:
            break

## 필요한 정보 추출하기 - 정규표현식 
- 평균평점: 사용자가 다른 리뷰에서 준 평점 합산한 것의 평균
- 평점 : 사용자가 해당 장소에 대하여 평가한 평점 
- 방문횟수
- 날짜
- 리뷰(한줄평): 빈칸이면 'None'


**~cleaner: 특정 표현 제외** 

In [7]:
# html 태그 제거
html_cleaner = re.compile('<.*?>')
#평점 이외 표현 제거
score_cleaner = re.compile(r'\d{4}.\d{2}.\d{3}'+'번째 방문영수증')


**~format : 특정표현 포함**


In [9]:
#평균 평점 추출용 : 평균평점+ 숫자
avg_score_format1 = re.compile('평균별점'+r'\d{1}.\d{1}') #평균평점<n.n> --> <n.n>
avg_score_format2 = re.compile('평균별점'+r'\d{1}')#평균평점<n> --> <n>
#날짜 추출용: YYYY.MM.DD
date_format = re.compile(r'\d{4}.\d{2}.\d{2}')
#방문횟수 추출용: <n>번째 방문영수증 --> <n> 
visit_time_format = re.compile(r'\d'+'번째 방문영수증') 

## take_info: 필요한 정보를 데이터프레임로 정리
- 리뷰, 방문횟수, 작성날짜, 평균평점, 평점

In [20]:
def take_info(soup):
    visitor_reviews_lst = [] #리뷰(한줄평)
    visit_times_lst = [] #방문횟수
    review_dates_lst = [] #작성날짜
    avg_score_lst = [] #평균평점
    score_lst = [] #평점


    for s in soup.find_all('div',{'class':'_1Z_GL'}):

        ##----------평균평점 추출
        personal_info = s.find('div',{'class':'_23Rml'})
        personal_info = re.sub(html_cleaner,"",str(personal_info))#html 태그 제거 

        try:#평균평점에 소수점이 있는 경우
            matchobj = avg_score_format1.search(str(personal_info))
            avg_score = matchobj.group()
            avg_score_lst.append(avg_score[4:])

        except:#평균평점이 정수인 경우
            matchobj = avg_score_format2.search(str(personal_info))
            avg_score = matchobj.group()
            avg_score_lst.append(avg_score[4:])


        ##-----------방문정보(평점, 날짜, 방문횟수) 추출
        visit_info = s.find('div',{'class':'_1ZcDn'})
        visit_info = re.sub(html_cleaner,"",str(visit_info)) #html 태그 제거 

        #날짜 추출
        matchobj = date_format.search(visit_info)
        rev_date = matchobj.group()
        review_dates_lst.append(rev_date)

        #방문횟수 추출
        matchobj = visit_time_format.search(visit_info)
        visit_num = matchobj.group()
        visit_times_lst.append(visit_num[0])

        #평점 추출
        score = re.sub(score_cleaner,"",visit_info)
        score_lst.append(score)


        ##-------------리뷰 추출
        try:
            review = s.find('div',{'class':'PVBo8'})
            review = re.sub(html_cleaner,"",str(review))
            visitor_reviews_lst.append(review)


        except:
            visitor_reviews_lst.append('None')
            
    #데이터 프레임으로 만들기 
    visit_reviews = pd.DataFrame({'평균평점':avg_score_lst,'평점':score_lst,'한줄평':visitor_reviews_lst,'작성날짜':review_dates_lst,'방문횟수':visit_times_lst})
        
    return visit_reviews

## 맛/서비스/만족도별 리뷰 크롤링(다예)

## 전체 크롤링 코드

In [91]:
def crawling(store_number,see_more_xpath):
    #크롬드라이버 구동하기 
    options = webdriver.ChromeOptions()
    options.add_argument('headless')#창 안 뜨게 설정
    review_driver = webdriver.Chrome('./chromedriver.exe',options=options)# 크롬 경로
    review_driver.implicitly_wait(3)
    
    #url 접속
    review_url = 'https://pcmap.place.naver.com/restaurant/'+store_number+'/home?from=map&amp;fromPanelNum=2&amp'
    review_driver.get(review_url)
    time.sleep(5)
    
    """홈에서 영업정보 크롤링하는 함수 호출(민주)"""

    #스크롤 다운 후 더보기 클릭
    scroll_click(review_driver, see_more_xpath)

    html = review_driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    #review_df에 필요한 정보 담기
    review_df = take_info(soup)
    
    """특징별 리뷰 뽑아오는 함수 호출(다예)"""

    return review_df

# 크롤링 실행 후 df에 저장

In [81]:
#지점 고유번호
store_number_lst = ['21726749', '34582591','1670762252']
store_name_lst = ['투썸플레이스 종로구청점','투썸플레이스 종각역점','투썸플레이스 종로알파빌딩점']

## 리뷰url마다 더보기란의 xpath경로 파악하기 

**내가 찾는 지점 리뷰url 접속**

In [ ]:
store_number = store_number_lst[0] 
options = webdriver.ChromeOptions()
options.add_argument('headless')#창 안 뜨게 설정
review_driver = webdriver.Chrome('./chromedriver.exe',options=options)# 크롬 경로
review_driver.implicitly_wait(3)

#url 접속
review_url = 'https://pcmap.place.naver.com/restaurant/'+store_number+'/home?from=map&amp;fromPanelNum=2&amp'
review_driver.get(review_url)
time.sleep(5)


**f12눌러서 방문자 리뷰 더보기란의 xpath 찾기<br>**

이런 태그들을 따라 클릭하다 보면 찾을 수 있음<br>
:#app-root > div > div > div.place_detail_wrapper > div:nth-child(5) > div > div.place_section._2Sq-S > **div._2kAri** > a

xpath예시:<br>
//*[@id="app-root"]/div/div/div[2]/div[5]/div/div[4]/div[2]/a<br>
//*[@id="app-root"]/div/div/div[2]/div[5]/div/div[8]/div[2]/a

## 크롤링한 후 전체 df로 합치기

In [85]:
#더보기 xapth 경로는 순서대로 list에 저장
see_more_xpath_lst = ['//*[@id="app-root"]/div/div/div[2]/div[5]/div/div[4]/div[2]/a','//*[@id="app-root"]/div/div/div[2]/div[5]/div/div[8]/div[2]/a','//*[@id="app-root"]/div/div/div[2]/div[5]/div/div[4]/div[2]/a']

In [86]:
review_all_df = pd.DataFrame()
for i in range(len(store_number_lst)):
    #i번째 지점의 크롤링 review_df 생성
    review_df = crawling(store_number_lst[i],see_more_xpath_lst[i])
    #review_df의 첫 열에 지점명 추가
    review_df.insert(0,'지점명',store_name_lst[i],allow_duplicates=False)
    #전체 df에 합치기
    review_all_df = pd.concat([review_all_df, review_df],axis=0)

In [88]:
review_all_df.to_excel('./방문자리뷰.xlsx', index=False)